# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``. 

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [2]:
from mxnet import ndarray as nd
import time

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details. 

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [29]:
A=nd.random.normal(shape=[4096,4096])
B=nd.random.normal(shape=[4096,4096])
t1=time.time()
C=nd.dot(A,B)
C.wait_to_read()
print('Direct dot-product time is: ', time.time()-t1)

t2=time.time()
for i in range(4096):
    C[:,1]=nd.dot(A,B[:,i])
C.wait_to_read()
print('Treat B as column vectors, time is: ', time.time()-t2)

t3=time.time()
for i in range(4096):
    for j in range(4096):
        C[i,j]=nd.dot(A[i,:],B[:,j])
C.wait_to_read()
print('Treat A, B as collection of vectors, time is ', time.time()-t3)


Direct dot-product time is:  1.853985071182251
Treat B as column vectors, time is:  12.81615686416626
Treat A, B as collection of vectors, time is  5011.84293961525


## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

1. We can prove B to be positive semidefinite by showing that there exists some matrix $Q$, s.t. $B=Q^\top Q$. Since $B=ADA^\top$ and $D$ is a diagonal matrix with nonnegative entries, we can set $Q=\sqrt{D}A^\top$, thus $Q^\top=A\sqrt{D}$. Hence $Q^\top Q=A\sqrt{D}\sqrt{D}A\top=ADA^\top=B$. By definition, we have proved  that $B$ is positive semidefinite.
2. When it comes to multiply B with another matrix or vector, it is better to use $B$. But if we want to calculate $B$ to its power (like $B\cdot B\cdot B\cdots$), then we can do faster with $A$ and $D$ if $m$ is much larger than $n$.

## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

In [1]:
import mxnet as mx
from mxnet import ndarray as nd
!nvidia-smi
x = nd.ones((2, 2), ctx=mx.gpu())
x

NVIDIA-SMI couldn't find nvml.dll library in your system. Please make sure that the NVIDIA Display Driver is properly installed and present in your system.
Please also try adding directory that contains nvml.dll to your system PATH.


MXNetError: [11:16:01] C:\ci\libmxnet_1533399150922\work\src\engine\threaded_engine.cc:318: Check failed: device_count_ > 0 (-1 vs. 0) GPU usage requires at least 1 GPU

## 4. NDArray and NumPy 

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [33]:
import numpy as np
c=np.ndarray([4096,1])
A=nd.random.normal(shape=[4096,4096])
B=nd.random.normal(shape=[4096,4096])
t1=time.time()
for i in range(4096):
    c[i]=nd.norm(nd.dot(A, B[:, i])).asnumpy()
print('Assign outcome to c_i directly: ', time.time()-t1)

d=nd.zeros(shape=[4096,1])
t2=time.time()
for i in range(4096):
    d[i]=nd.norm(nd.dot(A, B[:, i]))
c=d.asnumpy()
print('Assign to intermediate storage d: ', time.time()-t2)

Assign outcome to c_i directly:  16.207096815109253
Assign to intermediate storage d:  12.968188047409058


## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [14]:
# Random initialize some matrices A, B and C
A=nd.random.normal(4096,4096)
B=nd.random.normal(4096,4096)
C=nd.random.normal(4096,4096)

C[:]+=nd.elemwise_mul(A, B)

## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [ ]:
# Suppose we implement the calculation as a function
# The input parameters are vector x and power j, and the return value is the design matrix A
def design_mat(x, j):
    A=nd.zeros(20, 20)
    return A